In [4]:
# using os and zipfile library to prepare the dataset
import os
import zipfile
import json

# library for data processing
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

# library to make the recommendation system model
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# library for evaluate the machine learning model
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

In [5]:
# prepares the Kaggle credential environment
os.environ['KAGGLE_USERNAME'] = 'jayeshrmohanani'
os.environ['KAGGLE_KEY'] = '030582329579eb59561c5aeb1fd5f65e'

In [6]:
df = pd.read_csv("C:\\Users\jayes\OneDrive\Desktop\College\MSc\CHRIST Data Science\T2\Fullstack Web dev\CAC 2 - Project\games-dataset\Video_Games_Sales_as_at_22_Dec_2016.csv")
df.head()

,Game_Name,Platform,Release_Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.shape


(16719, 16)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game_Name     16717 non-null  object 
 1   Platform      16719 non-null  object 
 2   Release_Year  16450 non-null  float64
 3   Genre         16717 non-null  object 
 4   Publisher     16665 non-null  object 
 5   NA_Sales      16719 non-null  float64
 6   EU_Sales      16719 non-null  float64
 7   JP_Sales      16719 non-null  float64
 8   Other_Sales   16719 non-null  float64
 9   Global_Sales  16719 non-null  float64
 10  Critic_Score  8137 non-null   float64
 11  Critic_Count  8137 non-null   float64
 12  User_Score    10015 non-null  object 
 13  User_Count    7590 non-null   float64
 14  Developer     10096 non-null  object 
 15  Rating        9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [9]:
df.isna().sum()

Game_Name          2
Platform           0
Release_Year     269
Genre              2
Publisher         54
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
Global_Sales       0
Critic_Score    8582
Critic_Count    8582
User_Score      6704
User_Count      9129
Developer       6623
Rating          6769
dtype: int64

In [10]:
df.describe()

,Release_Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Count
count,16450.000000,16719.000000,16719.000000,16719.000000,16719.000000,16719.000000,8137.000000,8137.000000,7590.000000
mean,2006.487356,0.263330,0.145025,0.077602,0.047332,0.533543,68.967679,26.360821,162.229908
std,5.878995,0.813514,0.503283,0.308818,0.186710,1.547935,13.938165,18.980495,561.282326
min,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000,13.000000,3.000000,4.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000,60.000000,12.000000,10.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000,71.000000,21.000000,24.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,0.470000,79.000000,36.000000,81.000000
max,2020.000000,41.360000,28.960000,10.220000,10.570000,82.530000,98.000000,113.000000,10665.000000


In [11]:
df.drop(['Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Count'], axis=1, inplace=True)

In [12]:
#Check missing value
df[df['Game_Name'].isna()]

# Drop missing-value
for index in df[df['Game_Name'].isna()].index:
  df.drop(index, axis=0, inplace=True)

# make sure the missing value has been deleted
if(df['Game_Name'].isna().sum() == 0):
  print("There is no empty data in the Name column")
else:
  print("Missing value detected")

There is no empty data in the Name column


In [13]:
# use collections Counter to check the sum of each platform column element
platform_counter = Counter(df['Platform'])
platform_counter

# removes columns with counts less than 350
platform_less_than_350 = ['2600', '3DO', 'DC', 'GB', 'GEN', 'GG', 'N64','NES', 'NG',
                          'PCFX', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'WiiU', 'XOne']

df = df[~df['Platform'].isin(platform_less_than_350)]

# Check unique element
df['Platform'].unique()

array(['Wii', 'DS', 'X360', 'PS3', 'PS2', 'GBA', 'PS4', '3DS', 'PS', 'XB',
       'PC', 'PSP', 'GC', 'PSV'], dtype=object)

In [14]:
# Check missing value in Genre
df['Genre'].isna().sum()

np.int64(0)

In [15]:
# Check unique element
df['Genre'].unique()

array(['Sports', 'Racing', 'Platform', 'Misc', 'Simulation', 'Action',
       'Role-Playing', 'Puzzle', 'Shooter', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

In [16]:
# Check the sum of each unique element
genre_counter = Counter(df['Genre'])
genre_counter

# discard row with misc genre that is too complex
df = df[df['Genre'] != 'Misc']

# check dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13512 entries, 0 to 16718
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game_Name     13512 non-null  object 
 1   Platform      13512 non-null  object 
 2   Release_Year  13293 non-null  float64
 3   Genre         13512 non-null  object 
 4   Publisher     13485 non-null  object 
 5   NA_Sales      13512 non-null  float64
 6   EU_Sales      13512 non-null  float64
 7   JP_Sales      13512 non-null  float64
 8   Other_Sales   13512 non-null  float64
 9   User_Score    8844 non-null   object 
 10  Developer     8925 non-null   object 
 11  Rating        8792 non-null   object 
dtypes: float64(5), object(7)
memory usage: 1.3+ MB


In [17]:
# check missing-value
df['Publisher'].isna().sum()

np.int64(27)

In [18]:
# remove every riw with missing value
for index in df[df['Publisher'].isna()].index:
  df.drop(index, axis=0, inplace=True)

# recheck every missing value in the publisher column has been removed
if(df['Publisher'].isna().sum() == 0):
  print("There is no empty data in the Publisher column")
else:
  print("Missing value detected")

There is no empty data in the Publisher column


In [19]:
# Check unique element
df['Publisher'].unique()

array(['Nintendo', 'Take-Two Interactive', 'Sony Computer Entertainment',
       'Activision', 'Microsoft Game Studios', 'Bethesda Softworks',
       'Electronic Arts', 'Sega', 'SquareSoft', '505 Games', 'Ubisoft',
       'GT Interactive', 'Konami Digital Entertainment', 'Square Enix',
       'Sony Computer Entertainment Europe', 'Virgin Interactive',
       'LucasArts', 'Capcom', 'Warner Bros. Interactive Entertainment',
       'Universal Interactive', 'Eidos Interactive', 'Atari',
       'Vivendi Games', 'Enix Corporation', 'Hasbro Interactive',
       'Namco Bandai Games', 'THQ', 'Fox Interactive',
       'Acclaim Entertainment', 'Disney Interactive Studios',
       'Codemasters', 'Majesco Entertainment', 'Red Orb', 'Level 5',
       'Midway Games', 'JVC', 'Deep Silver', 'NCSoft', '989 Studios',
       'UEP Systems', 'Maxis', 'Tecmo Koei', 'ASCII Entertainment',
       'Valve Software', 'Unknown', 'Valve', 'Hello Games', 'D3Publisher',
       'Activision Value', 'Infogrames', 'Red S

In [20]:
# Check missing value
df['Genre'].isna().sum()

np.int64(0)

In [21]:
# Check unique element
df['Genre'].unique()

array(['Sports', 'Racing', 'Platform', 'Simulation', 'Action',
       'Role-Playing', 'Puzzle', 'Shooter', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

In [22]:
# Check the sum of each unique element
genre_counter = Counter(df['Genre'])
genre_counter

Counter({'Action': 3076,
         'Sports': 2105,
         'Role-Playing': 1358,
         'Adventure': 1225,
         'Shooter': 1180,
         'Racing': 1131,
         'Simulation': 831,
         'Platform': 737,
         'Fighting': 716,
         'Strategy': 621,
         'Puzzle': 505})

In [23]:
# discard row with misc genre that is too complex
df = df[df['Genre'] != 'Misc']

In [24]:
# check dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13485 entries, 0 to 16718
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game_Name     13485 non-null  object 
 1   Platform      13485 non-null  object 
 2   Release_Year  13285 non-null  float64
 3   Genre         13485 non-null  object 
 4   Publisher     13485 non-null  object 
 5   NA_Sales      13485 non-null  float64
 6   EU_Sales      13485 non-null  float64
 7   JP_Sales      13485 non-null  float64
 8   Other_Sales   13485 non-null  float64
 9   User_Score    8837 non-null   object 
 10  Developer     8917 non-null   object 
 11  Rating        8786 non-null   object 
dtypes: float64(5), object(7)
memory usage: 1.3+ MB


In [25]:
# check missing-value
df['Publisher'].isna().sum()

np.int64(0)

In [26]:
# remove every riw with missing value
for index in df[df['Publisher'].isna()].index:
  df.drop(index, axis=0, inplace=True)

# recheck every missing value in the publisher column has been removed
if(df['Publisher'].isna().sum() == 0):
  print("There is no empty data in the Publisher column")
else:
  print("Missing value detected")

There is no empty data in the Publisher column


In [27]:
# Check unique element
df['Publisher'].unique()

array(['Nintendo', 'Take-Two Interactive', 'Sony Computer Entertainment',
       'Activision', 'Microsoft Game Studios', 'Bethesda Softworks',
       'Electronic Arts', 'Sega', 'SquareSoft', '505 Games', 'Ubisoft',
       'GT Interactive', 'Konami Digital Entertainment', 'Square Enix',
       'Sony Computer Entertainment Europe', 'Virgin Interactive',
       'LucasArts', 'Capcom', 'Warner Bros. Interactive Entertainment',
       'Universal Interactive', 'Eidos Interactive', 'Atari',
       'Vivendi Games', 'Enix Corporation', 'Hasbro Interactive',
       'Namco Bandai Games', 'THQ', 'Fox Interactive',
       'Acclaim Entertainment', 'Disney Interactive Studios',
       'Codemasters', 'Majesco Entertainment', 'Red Orb', 'Level 5',
       'Midway Games', 'JVC', 'Deep Silver', 'NCSoft', '989 Studios',
       'UEP Systems', 'Maxis', 'Tecmo Koei', 'ASCII Entertainment',
       'Valve Software', 'Unknown', 'Valve', 'Hello Games', 'D3Publisher',
       'Activision Value', 'Infogrames', 'Red S

In [28]:
# Check the unknown element
df[df['Publisher'] == 'Unknown']

,Game_Name,Platform,Release_Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score,Developer,Rating
944,Gran Turismo Concept 2001 Tokyo,PS2,2001.0,Racing,Unknown,0.00,1.10,0.42,0.33,NaN,NaN,NaN
1650,NASCAR Thunder 2003,PS2,NaN,Racing,Unknown,0.60,0.46,0.00,0.16,8.7,EA Sports,E
2108,Suikoden III,PS2,NaN,Role-Playing,Unknown,0.29,0.23,0.38,0.08,7.7,KCET,T
2224,Teenage Mutant Ninja Turtles,GBA,2003.0,Action,Unknown,0.67,0.25,0.00,0.02,8.8,Konami,E
2321,Blitz: The League,PS2,2005.0,Sports,Unknown,0.74,0.03,0.00,0.12,8,Midway,M
...,...,...,...,...,...,...,...,...,...,...,...,...
16558,"Horse Life 4: My Horse, My Friend, My Champion",3DS,2015.0,Action,Unknown,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16638,The Treasures of Mystery Island 3 Pack - Save ...,PC,2011.0,Puzzle,Unknown,0.01,0.00,0.00,0.00,NaN,NaN,NaN
16653,Real Crimes: The Unicorn Killer,DS,2011.0,Puzzle,Unknown,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16706,STORM: Frontline Nation,PC,2011.0,Strategy,Unknown,0.00,0.01,0.00,0.00,7.2,SimBin,E10+


In [29]:
# remove row with the unknown publisher
for index in df[df['Publisher'] == 'Unknown'].index:
  df.drop(index, axis=0, inplace=True)

In [30]:
# check missing value
df['Release_Year'].isna().sum()

np.int64(115)

In [31]:
# remove missing value
for index in df[df['Release_Year'].isna()].index:
  df.drop(index, axis=0, inplace=True)

# ensure the missing value has been removed
if(df['Release_Year'].isna().sum() == 0):
  print("There is no empty data in Release_Year column")
else:
  print("Missing value detected")

There is no empty data in Release_Year column


In [32]:
# check unique element
df['Release_Year'].unique()

array([2006., 2008., 2009., 2005., 2007., 2013., 2004., 2002., 2010.,
       2001., 2011., 2015., 2012., 2014., 1997., 1999., 2016., 2003.,
       1998., 1996., 2000., 1995., 1994., 1992., 2020., 2017., 1985.,
       1988.])

In [33]:
# change column type to string because it is categorical
df['Release_Year'] = df['Release_Year'].astype('str')

# check missing value of the whole data
df.isna().sum()

Game_Name          0
Platform           0
Release_Year       0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
User_Score      4559
Developer       4496
Rating          4615
dtype: int64

In [34]:
# remove the missing value in the user score, developer, and rating columns
for index in df[df['User_Score'].isna()].index:
  df.drop(index, axis=0, inplace=True)

for index in df[df['Developer'].isna()].index:
  df.drop(index, axis=0, inplace=True)

for index in df[df['Rating'].isna()].index:
  df.drop(index, axis=0, inplace=True)

# check that all missing values in the dataset have been removed
if df.isna().sum().sum() == 0:
  print('Dataset cleaned')
else:
  print('Missing value detected')

Dataset cleaned


In [35]:
# check unique element
df['User_Score'].unique()

array(['8', '8.3', '8.5', '8.4', '8.6', '7.7', '7.4', '8.2', '9', '8.1',
       '8.7', '7.1', '3.4', '6.3', '5.3', '4.8', '3.2', '8.9', '6.4',
       '7.8', '7.9', '7.5', '2.6', '7.2', '9.2', '7', '4.3', '6.6', '7.6',
       '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '7.3', '9.4', '6.8',
       '6.1', '6.7', '4', '5.4', '4.9', '4.5', '9.3', '4.2', '3.7', '5.8',
       '5.6', '5.9', '3.9', '5.5', '6.2', '5.2', '6', '4.1', '4.7', '4.4',
       '5.1', '3.5', '2.5', '3', '3.1', '2.9', '2.7', '2.2', '2', '4.6',
       '9.5', '2.1', '3.6', '2.8', '3.3', '1.8', '3.8', '0', '1.6', '9.6',
       '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2', '2.3', '1.3',
       '0.2', '0.5', '0.6', '1.4', '0.9', '1.9', '1', '9.7'], dtype=object)

In [36]:
# check tbd element in User_Score column
df[df['User_Score'] == 'tbd']

# remove row with tbd user score
for index in df[df['User_Score'] == 'tbd'].index:
  df.drop(index, axis=0, inplace=True)

# change the user score data type to float as a numerical feature
df['User_Score'] = df['User_Score'].astype('float')

In [37]:
# check the number of different elements in the developer column
df['Developer'].nunique()

1267

In [38]:
# because the number of different elements is too much and the column is categorical then the column is discarded
df.drop('Developer', axis=1, inplace=True)

In [39]:
# check unique element
df['Rating'].unique()

array(['E', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

In [40]:
df.duplicated().sum()

np.int64(0)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6662 entries, 0 to 16700
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game_Name     6662 non-null   object 
 1   Platform      6662 non-null   object 
 2   Release_Year  6662 non-null   object 
 3   Genre         6662 non-null   object 
 4   Publisher     6662 non-null   object 
 5   NA_Sales      6662 non-null   float64
 6   EU_Sales      6662 non-null   float64
 7   JP_Sales      6662 non-null   float64
 8   Other_Sales   6662 non-null   float64
 9   User_Score    6662 non-null   float64
 10  Rating        6662 non-null   object 
dtypes: float64(5), object(6)
memory usage: 624.6+ KB


In [42]:
df = df.reset_index(drop=True)
df

df.describe()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score
count,6662.000000,6662.000000,6662.000000,6662.000000,6662.000000
mean,0.370982,0.224319,0.059917,0.080369,7.165596
std,0.925752,0.666564,0.275964,0.267998,1.492732
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.060000,0.020000,0.000000,0.010000,6.500000
50%,0.140000,0.050000,0.000000,0.020000,7.500000
75%,0.370000,0.200000,0.010000,0.070000,8.200000
max,41.360000,28.960000,6.500000,10.570000,9.700000


In [43]:
# save game names on new dataframe
df_game_name = pd.DataFrame({'Game': df['Game_Name']}).reset_index(drop=True)
df_game_name.head()

,Game
0,Wii Sports
1,Mario Kart Wii
2,Wii Sports Resort
3,New Super Mario Bros.
4,New Super Mario Bros. Wii


In [44]:
# use name column as index
df.set_index('Game_Name', inplace=True)
df.head()

,Platform,Release_Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score,Rating
Game_Name,,,,,,,,,,
Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,8.0,E
Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,8.3,E
Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,8.0,E
New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,8.5,E
New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,8.4,E


In [45]:
# select all columns with datatype object
column_object = df.dtypes[df.dtypes == 'object'].keys()
column_object

Index(['Platform', 'Release_Year', 'Genre', 'Publisher', 'Rating'], dtype='object')

In [46]:
# convert category data to one-hot encoding
one_hot_label = pd.get_dummies(df[column_object])
one_hot_label.head(3)

,Platform_3DS,Platform_DS,Platform_GBA,Platform_GC,Platform_PC,Platform_PS,Platform_PS2,Platform_PS3,Platform_PS4,Platform_PSP,...,Publisher_id Software,Publisher_inXile Entertainment,Rating_AO,Rating_E,Rating_E10+,Rating_EC,Rating_K-A,Rating_M,Rating_RP,Rating_T
Game_Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Mario Kart Wii,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Wii Sports Resort,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [47]:
# delete column with data type object
df.drop(column_object,axis=1,inplace=True)
df.head()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score
Game_Name,,,,,
Wii Sports,41.36,28.96,3.77,8.45,8.0
Mario Kart Wii,15.68,12.76,3.79,3.29,8.3
Wii Sports Resort,15.61,10.93,3.28,2.95,8.0
New Super Mario Bros.,11.28,9.14,6.50,2.88,8.5
New Super Mario Bros. Wii,14.44,6.94,4.70,2.24,8.4


In [48]:
# unify one-hot encoding data with whole data
df = pd.concat([df,one_hot_label],axis=1)
df.head()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score,Platform_3DS,Platform_DS,Platform_GBA,Platform_GC,Platform_PC,...,Publisher_id Software,Publisher_inXile Entertainment,Rating_AO,Rating_E,Rating_E10+,Rating_EC,Rating_K-A,Rating_M,Rating_RP,Rating_T
Game_Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,41.36,28.96,3.77,8.45,8.0,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Mario Kart Wii,15.68,12.76,3.79,3.29,8.3,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Wii Sports Resort,15.61,10.93,3.28,2.95,8.0,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
New Super Mario Bros.,11.28,9.14,6.50,2.88,8.5,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
New Super Mario Bros. Wii,14.44,6.94,4.70,2.24,8.4,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [49]:
# select all numeric column
column_numeric = list(df.dtypes[df.dtypes == 'float64'].keys())
column_numeric


['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'User_Score']

In [50]:
# MinMaxScaler initiation
scaler = MinMaxScaler()

In [51]:
# numerical column data standardization
scaled = scaler.fit_transform(df[column_numeric])

In [52]:
# scaled the data
i=0
for column in column_numeric:
    df[column] = scaled[:,i]
    i += 1

# check the result of the normalized data
df.head()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score,Platform_3DS,Platform_DS,Platform_GBA,Platform_GC,Platform_PC,...,Publisher_id Software,Publisher_inXile Entertainment,Rating_AO,Rating_E,Rating_E10+,Rating_EC,Rating_K-A,Rating_M,Rating_RP,Rating_T
Game_Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,1.000000,1.000000,0.580000,0.799432,0.824742,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Mario Kart Wii,0.379110,0.440608,0.583077,0.311258,0.855670,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
Wii Sports Resort,0.377418,0.377417,0.504615,0.279092,0.824742,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
New Super Mario Bros.,0.272727,0.315608,1.000000,0.272469,0.876289,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
New Super Mario Bros. Wii,0.349130,0.239641,0.723077,0.211921,0.865979,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [53]:
df.describe()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,User_Score
count,6662.000000,6662.000000,6662.000000,6662.000000,6662.000000
mean,0.008970,0.007746,0.009218,0.007604,0.738721
std,0.022383,0.023017,0.042456,0.025355,0.153890
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.001451,0.000691,0.000000,0.000946,0.670103
50%,0.003385,0.001727,0.000000,0.001892,0.773196
75%,0.008946,0.006906,0.001538,0.006623,0.845361
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [54]:
# Model initiation
model = NearestNeighbors(metric='euclidean')

In [55]:
# Fit model to the data
model.fit(df)

NearestNeighbors(metric='euclidean')

In [56]:
# Create function to get the game recommendation
def GameRecommended(gamename:str, recommended_games:int=6):
  print(f'If user like playing Game: \n{gamename[0]}\n5 Game that the user might like to play:')
  # Looking for the game with the highest similarity to the game that user play
  distances, neighbors = model.kneighbors(df.loc[gamename],n_neighbors=recommended_games)
  # Input the recommended game into the list
  similar_game = []
  for gamename in df_game_name.loc[neighbors[0][:]].values:
    similar_game.append(gamename[0])
  # Input the distance score into the list
  similar_distance = []
  for distance in distances[0]:
    similar_distance.append(f"{round(100-distance, 2)}%")
  # Return a dataframe with the most recommended game
  return pd.DataFrame(data = {"Game" : similar_game[1:], "Similarity" : similar_distance[1:]})

In [57]:
# Give the recommendation to the selected game
GameRecommended(df_game_name.loc[111])

If user like playing Game: 
Final Fantasy IX
5 Game that the user might like to play:


C:\Users\Jayesh\AppData\Local\Temp\ipykernel_7276\1073923334.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'If user like playing Game: \n{gamename[0]}\n5 Game that the user might like to play:')


,Game,Similarity
0,Final Fantasy VIII,98.58%
1,Final Fantasy Tactics,98.57%
2,Xenogears,98.55%
3,Tales of Destiny II,98.55%
4,Chrono Cross,98.55%


In [58]:
# Calculate the cosine similarity of the dataframe
cosine_sim = cosine_similarity(df)

In [59]:
# Keep the result of the calculation dataframe
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_game_name['Game'], columns=df_game_name['Game'])
cosine_sim_df.head(3)

Game,Wii Sports,Mario Kart Wii,Wii Sports Resort,New Super Mario Bros.,New Super Mario Bros. Wii,Mario Kart DS,Wii Fit,Wii Fit Plus,Grand Theft Auto V,Grand Theft Auto: San Andreas,...,Trine,Karnaaj Rally,Hospital Tycoon,Ben 10 Omniverse 2,Bookworm Deluxe,E.T. The Extra-Terrestrial,Mortal Kombat: Deadly Alliance,Worms 2,Metal Gear Solid V: Ground Zeroes,Breach
Game,,,,,,,,,,,,,,,,,,,,,
Wii Sports,1.000000,0.681225,0.806006,0.655580,0.652319,0.504198,0.774904,0.769941,0.217456,0.238192,...,0.099181,0.251738,0.048428,0.064216,0.382273,0.181952,0.105425,0.098121,0.092749,0.072453
Mario Kart Wii,0.681225,1.000000,0.686753,0.541707,0.686951,0.682935,0.677577,0.669040,0.180855,0.184008,...,0.118648,0.455775,0.057913,0.076806,0.280582,0.211163,0.126127,0.117379,0.110950,0.086671
Wii Sports Resort,0.806006,0.686753,1.000000,0.530365,0.838024,0.520196,0.835151,0.995444,0.172429,0.176879,...,0.282934,0.295104,0.056737,0.075247,0.280941,0.213287,0.123572,0.114998,0.108700,0.084915


In [60]:
# Create function to get the game recommendation
def CosineGameRecommended(gamename:str, recommended_games:int=5):
  print(f'If user like playing Game: \n{gamename[0]}\n5 Game that the user might like to play:')
  # Look up the unique value of the game the user likes in the cosine sim dataframe row
  # The unique value (arr) is returned in an ordered form from small to large 
  arr, ind = np.unique(cosine_sim_df.loc[gamename[0]], return_index=True)
  # Input similar game names from the second-last index to the nth-last index
  similar_game = []
  for index in ind[-(recommended_games+1):-1]:
    similar_game.append(df_game_name.loc[index][0])
  # Input the cosine scores of similar games starting from the second-last index to the nth-last index.
  cosine_score = []
  for score in arr[-(recommended_games+1):-1]:
    cosine_score.append(score)
  # Return a dataframe with the most recommended game
  return pd.DataFrame(data = {"Game" : similar_game, "Cosine Similarity" : cosine_score}).sort_values(by='Cosine Similarity',ascending=False)

In [75]:
# provides recommendations with cosine similarity on selected games
CosineGameRecommended(df_game_name.loc[487])

If user like playing Game: 
Kingdom Hearts: Birth by Sleep
5 Game that the user might like to play:


C:\Users\Jayesh\AppData\Local\Temp\ipykernel_7276\2992978366.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'If user like playing Game: \n{gamename[0]}\n5 Game that the user might like to play:')
C:\Users\Jayesh\AppData\Local\Temp\ipykernel_7276\2992978366.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arr, ind = np.unique(cosine_sim_df.loc[gamename[0]], return_index=True)
C:\Users\Jayesh\AppData\Local\Temp\ipykernel_7276\2992978366.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Dat

,Game,Cosine Similarity
4,Tactics Ogre: Let Us Cling Together,0.827399
3,Lufia: Curse of the Sinistrals,0.824357
2,Final Fantasy Anniversary Edition,0.823523
1,Final Fantasy IV: The Complete Collection,0.823028
0,Kingdom Hearts Re:coded,0.820686
